In [ ]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_core.messages import HumanMessage
import os

from dotenv import load_dotenv

load_dotenv()
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")
model = ChatHuggingFace(llm=llm)


/home/nbx/Documents/agentic/DocuMind/Backend/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
loader = PyPDFLoader("test.pdf")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)

print(f"📄 Prepared {len(chunks)} chunks")

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_name="agent_test_collection"
)

print("vectore store created")

📄 Prepared 13 chunks
vectore store created


In [3]:
@tool
def search_documents(query: str) -> str:
    """
    Search the PDF content and return relevant text chunks.
    """
    results = vectorstore.similarity_search(query, k=3)

    if not results:
        return "No relevant information found in the documents."

    return "\n\n".join(
        f"- {doc.page_content}"
        for doc in results
    )

In [4]:
agent = create_agent(
    model,
    tools = [search_documents],
    system_prompt="You are a document question-answering assistant. Answer the user's question using ONLY the provided document .Do not make up information. Keep answers short and clear."
)

In [9]:
result = agent.invoke(
    {"messages": [HumanMessage("explain the nagle alghorithme")]}
)
ai_message = result["messages"][1]  # second element in the list
print(ai_message.content)




The Nagle algorithm, also known as TCP Tidal Window algorithm or TCP No-Copy algorithm, is a method used to improve network efficiency in Transmission Control Protocol (TCP) for data transmission by delaying the sending of small packets until a certain amount of data is accumulated in order to reduce the number of packets sent and thus decrease the overhead of transmitting small packets. This algorithm helps in the avoidance of sending packets that have less data and results in a more efficient use of the network bandwidth. It works by buffering the outgoing packets and sending them in a larger chunk, which is known as a "sliding window", thus there is less overhead for transmitting the packet headers and lowering the number of packets transmitted, hence less network delay. The Nagle algorithm is disabled by default in most modern operating systems because of the prevalence of fast networks and the availability of larger buffers, but it is still used in some other applications that r